<a href="https://colab.research.google.com/github/viniciostec22/Introducao_Redes_Neurais/blob/main/atividade_01_redes_neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torchvision
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
import tqdm.notebook as tq
import os


In [ ]:
BASE_DIR = '/content/drive/MyDrive/Mestrado_CNN/Dataset_banana/resized/resized'
# DATA_DIR = '/content/drive/MyDrive/datasets/cifar10'
CATEGORIES = ['BLACK_SIGATOKA', 'HEALTHY_LEAVES', 'PANAMA_DISEASE', 'YELLOW_SIGATOKA']
def get_path(relpath):
  return os.path.join(BASE_DIR, relpath)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Rodando na {device}")

Rodando na cuda


# Obeter valores de noramalização

In [ ]:
# from torch.utils.data import Subset

# # Defina a transformação sem normalização
# transformacao_sem_normalizacao = transforms.Compose([
#     transforms.Resize((150, 150)),
#     transforms.ToTensor(),
# ])

# # Crie um conjunto de dados sem normalização
# conjunto_dados_sem_normalizacao = datasets.ImageFolder(root=BASE_DIR, transform=transformacao_sem_normalizacao)

# # Obtenha um índice aleatório
# indices_aleatorios = torch.randperm(len(conjunto_dados_sem_normalizacao))[:tamanho_amostra]

# # Crie uma subclasse do conjunto de dados usando os índices aleatórios
# amostra_aleatoria = Subset(conjunto_dados_sem_normalizacao, indices_aleatorios)

# # Calcule as médias e desvios padrão usando a amostra
# medias = torch.stack([imagem.mean((1, 2)) for imagem, _ in amostra_aleatoria]).mean(0)
# desvios_padrao = torch.stack([imagem.std((1, 2)) for imagem, _ in amostra_aleatoria]).mean(0)

# print("Médias:", medias)
# print("Desvios Padrão:", desvios_padrao)




Médias: tensor([0.4308, 0.5051, 0.3152])
Desvios Padrão: tensor([0.1718, 0.1682, 0.1632])


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transformações que serão aplicadas às imagens
transformacoes = transforms.Compose(
    [
    transforms.Resize((150, 150)),  # Ajuste o tamanho conforme necessário
    transforms.ToTensor(),
    transforms.Normalize(
        (0.4308, 0.5051, 0.3152),
        (0.1718, 0.1682, 0.1632),

        )
    ]
)

# Crie um conjunto de dados personalizado
dataset = datasets.ImageFolder(root=BASE_DIR, transform=transformacoes)

# Divida o conjunto de dados em treino, validação e teste
tamanho_treino = int(0.7 * len(dataset))  # 70% para treino
tamanho_validacao = int(0.15 * len(dataset))  # 15% para validação
tamanho_teste = len(dataset) - tamanho_treino - tamanho_validacao  # O restante para teste

conjunto_treino, conjunto_validacao, conjunto_teste = random_split(dataset, [tamanho_treino, tamanho_validacao, tamanho_teste])


In [ ]:
# Crie DataLoader para cada conjunto
batch_size = 64

loader_treino = DataLoader(conjunto_treino, batch_size=batch_size, shuffle=True)
loader_validacao = DataLoader(conjunto_validacao, batch_size=batch_size, shuffle=True)
loader_teste = DataLoader(conjunto_teste, batch_size=batch_size, shuffle=True)

In [ ]:
def visualizar_tamanhos_dataset(loader, banana):
    print(f"Tamanhos do conjunto de dados {banana}:")
    print(f"Número total de amostras: {len(loader.dataset)}")
    print(f"Número de lotes por época: {len(loader)}")
    print(f"Tamanho do lote: {loader.batch_size}")

# Exemplo de uso:
visualizar_tamanhos_dataset(loader_treino, "treino")
visualizar_tamanhos_dataset(loader_validacao, "validação")
visualizar_tamanhos_dataset(loader_teste, "teste")



Tamanhos do conjunto de dados treino:
Número total de amostras: 1748
Número de lotes por época: 28
Tamanho do lote: 64
Tamanhos do conjunto de dados validação:
Número total de amostras: 374
Número de lotes por época: 6
Tamanho do lote: 64
Tamanhos do conjunto de dados teste:
Número total de amostras: 376
Número de lotes por época: 6
Tamanho do lote: 64


In [ ]:
dataset = datasets.ImageFolder(root=BASE_DIR, transform=transformacoes)


In [ ]:
class ConvolutionalModel(nn.Module):
  def __init__(self, dropout_prob=0.5):
      super().__init__()
      self.convlayers = nn.Sequential(
        nn.Conv2d(3, 256, kernel_size=(11, 11)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(dropout_prob),

        nn.Conv2d(256, 128, kernel_size=(9, 9)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(dropout_prob),

        nn.Conv2d(128, 64, kernel_size=(5, 5)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        # nn.Conv2d(64, 32, kernel_size=(3, 3)),
        # nn.ReLU(),
        # nn.MaxPool2d(2, 2),

      )
      self._compute_linear_input_size()
      self.linearlayers = nn.Sequential(
            nn.Linear(self.linear_input_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 4),
            nn.Softmax(dim=1)  # Adicionando softmax
        )

  def _compute_linear_input_size(self):
        # Crie uma entrada de exemplo e calcule o tamanho
        # antes de criar as camadas lineares
        x = torch.randn(1, 3, 150, 150)
        x = self.convlayers(x)
        x = torch.flatten(x, 1)
        self.linear_input_size = x.size(1)

  def forward(self, x):
        x = self.convlayers(x)
        x = torch.flatten(x, 1)
        return self.linearlayers(x)

In [ ]:
convmodel = ConvolutionalModel().to(device)

In [ ]:
conv_optimizer = torch.optim.SGD(convmodel.parameters(), lr=0.001)
convlossfunc = nn.CrossEntropyLoss()

In [ ]:
epochs = 31

In [ ]:

# import torch
# import torch.nn as nn

# def treinar_modelo(modelo, loader_treino, loader_validacao, optimizer, loss_func, num_epochs=epochs):


#     for epoch in range(num_epochs):
#         modelo.train()
#         for imagens, labels in loader_treino:
#             imagens, labels = imagens.to(device), labels.to(device)

#             optimizer.zero_grad()
#             saidas = modelo(imagens)
#             perda = loss_func(saidas, labels)
#             perda.backward()
#             optimizer.step()

#         # Avaliação no conjunto de validação
#         modelo.eval()
#         acuracia_total = 0.0
#         total_samples = 0

#         with torch.no_grad():
#             for imagens, labels in loader_validacao:
#                 imagens, labels = imagens.to(device), labels.to(device)

#                 saidas = modelo(imagens)
#                 _, previsto = torch.max(saidas, 1)

#                 acuracia_total += torch.sum(previsto == labels).item()
#                 total_samples += labels.size(0)

#         acuracia = acuracia_total / total_samples
#         print(f"Época {epoch+1}/{num_epochs}, Perda: {perda.item():.4f}, Acurácia de Validação: {acuracia:.4f}")

#     print("Treinamento concluído.")


In [ ]:
# treinar_modelo(convmodel, loader_treino, loader_validacao, conv_optimizer, convlossfunc, num_epochs=epochs)


In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

def avaliar_modelo(modelo, loader, dispositivo):
    modelo.eval()
    acuracia_total = 0.0
    total_samples = 0
    perda_total = 0.0

    criterio = nn.CrossEntropyLoss()

    with torch.no_grad():
        for imagens, labels in loader:
            imagens, labels = imagens.to(dispositivo), labels.to(dispositivo)

            saidas = modelo(imagens)
            perda_total += criterio(saidas, labels).item()

            _, previsto = torch.max(saidas, 1)

            acuracia_total += torch.sum(previsto == labels).item()
            total_samples += labels.size(0)

    acuracia = acuracia_total / total_samples
    perda_media = perda_total / len(loader)

    return acuracia, perda_media

def treinar_avaliar_modelo(modelo, loader_treino, loader_validacao, loader_teste, optimizer, loss_func, num_epochs=epochs):

    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo = modelo.to(dispositivo)

    criterio = nn.CrossEntropyLoss()
    otimizador = optimizer

   # ...
    perda_treino_lista = []
    perda_validacao_lista = []
    perda_teste_lista = []
    acuracia_validacao_lista = []
    acuracia_teste_lista = []

    for epoch in range(num_epochs):
        modelo.train()
        progress_bar = tqdm(loader_treino, desc=f"Época {epoch+1}/{num_epochs}", leave=False)
        perda_total_epoch = 0.0  # Inicializar a perda total da época

        for imagens, labels in progress_bar:
            imagens, labels = imagens.to(dispositivo), labels.to(dispositivo)

            otimizador.zero_grad()
            saidas = modelo(imagens)
            perda = criterio(saidas, labels)
            perda.backward()
            otimizador.step()

            progress_bar.set_postfix({"Perda Treino": perda.item()})

            perda_total_epoch += perda.item()  # Acumular perda durante a época

        perda_media = perda_total_epoch / len(loader_treino)  # Calcular a média da perda
        perda_treino_lista.append(perda_media)

        # Avaliação no conjunto de validação
        acuracia_validacao, perda_validacao = avaliar_modelo(modelo, loader_validacao, dispositivo)
        perda_validacao_lista.append(perda_validacao)
        acuracia_validacao_lista.append(acuracia_validacao)

        # Avaliação no conjunto de teste
        acuracia_teste, perda_teste = avaliar_modelo(modelo, loader_teste, dispositivo)
        perda_teste_lista.append(perda_teste)
        acuracia_teste_lista.append(acuracia_teste)

        print(f"Época {epoch+1}/{num_epochs}, Perda Treino: {perda_media:.4f}, Perda Validação: {perda_validacao:.4f}, Perda Teste: {perda_teste:.4f}, Acurácia Validação: {acuracia_validacao:.4f}, Acurácia Teste: {acuracia_teste:.4f}")
  # ...



    return perda_treino_lista, perda_validacao_lista, perda_teste_lista, acuracia_validacao_lista, acuracia_teste_lista

# Exemplo de uso:
perda_treino, perda_validacao, perda_teste, acuracia_validacao, acuracia_teste = treinar_avaliar_modelo(convmodel, loader_treino, loader_validacao, loader_teste, conv_optimizer, convlossfunc, num_epochs=epochs)


KeyboardInterrupt: ignored

In [ ]:
print(len(perda_treino))
print(len(perda_validacao))
print(len(perda_teste))
print(len(acuracia_validacao))
print(len(acuracia_teste))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
num_epochs = epochs
def plotar_graficos(perda_treino, perda_validacao, perda_teste, acuracia_validacao, acuracia_teste, num_epochs):
    # Gráfico de perda
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs + 1), perda_treino, label='Treino')
    plt.plot(range(1, num_epochs + 1), perda_validacao, label='Validação')
    plt.plot(range(1, num_epochs + 1), perda_teste, label='Teste')
    plt.title('Perda ao Longo das Épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.show()
    plt.subplots_adjust(hspace=0.5)

    # Gráfico de acurácia
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs + 1), acuracia_validacao, label='Validação')
    plt.plot(range(1, num_epochs + 1), acuracia_teste, label='Teste')
    plt.title('Acurácia ao Longo das Épocas')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.show()
    plt.subplots_adjust(hspace=0.5)

def plotar_matriz_confusao(modelo, loader, dispositivo):
    modelo.eval()
    predicoes = []
    alvos = []

    with torch.no_grad():
        for imagens, labels in loader:
            imagens, labels = imagens.to(dispositivo), labels.to(dispositivo)

            saidas = modelo(imagens)
            _, previsto = torch.max(saidas, 1)

            predicoes.extend(previsto.cpu().numpy())
            alvos.extend(labels.cpu().numpy())

    matriz_confusao = confusion_matrix(alvos, predicoes)
    classes = [str(i) for i in range(matriz_confusao.shape[0])]

    plt.figure(figsize=(8, 8))
    sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Matriz de Confusão')
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.show()
    plt.subplots_adjust(hspace=0.5)
# Exemplo de uso:
plotar_graficos(perda_treino, perda_validacao, perda_teste, acuracia_validacao, acuracia_teste, num_epochs)
plotar_matriz_confusao(convmodel, loader_teste, device)
